# Analysis of spectral data

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import glob

In [ ]:
project_path = './'

data_path   = project_path + '../data/cospec_analysis/'

graphs_path = project_path + '../graphs/'

### Functions

In [ ]:
def load_all_cospec(directory, silent=False):
    if (not silent):
        print('  - Cospectra files:')
        
    file_list = sorted(glob.glob(directory + '**/*csv', recursive=True))
    data_list = []
    for idx, fn in enumerate(file_list):
        print('    -', fn.split('\\')[-1])
        # Read file
        temp = pd.read_csv(fn, skiprows=11, na_values=-9999)
        # Append timestamp
        timestamp_fn = fn.split('\\')[-1].split('_')[0]
        temp['DateTime'] = pd.to_datetime(timestamp_fn, format='%Y%m%d-%H%M')
        # shift column 'timestamp' to first position
        col = temp.pop('DateTime')
        temp.insert(0, 'DateTime', col)
        # Append to list of dfs
        data_list.append(temp)
    # Combine all the read data
    df = pd.concat(data_list, axis=0, ignore_index=True)
    # Make the timestamp the middle of the halfhour
    df['DateTime'] = df['DateTime'] + pd.Timedelta(minutes=15)
    return(df)

def add_ecosystem(df):
    df = df.copy()
    # Add ecosystem
    df['Ecosystem'] = np.nan
    df.loc[df['DateTime'] < '2019-07-17', 'Ecosystem'] = 'Desert background'
    df.loc[df['DateTime'] >= '2019-07-17', 'Ecosystem'] = 'PV field'
    # shift column 'timestamp' to first position
    col = df.pop('Ecosystem')
    df.insert(0, 'Ecosystem', col)
    col = df.pop('DateTime')
    df.insert(0, 'DateTime', col)
    return(df)

In [ ]:
print('Loading data...')
df = load_all_cospec(data_path)
df = add_ecosystem(df)

df['halfhour'] = df['DateTime'].dt.strftime('%H:%M')
print('Done...')

In [ ]:
display(df)

### Make figures

In [ ]:
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
# Colours
cbPalette = ['#E69F00', '#0072B2', '#939393', '#CC00CC', '#009E73', '#D55E00', '#CC79A7', '#FF3300', '#F0E442', '#56B4E9']

In [ ]:
plt = ggplot(df)
plt = plt + geom_point(aes(x='natural_frequency', y='f_nat*cospec(w_u)/cov(w_u)', colour='Ecosystem'), size=0.5)
plt = plt + theme_bw()
plt = plt + labs(x='Frequency', y='Cospectra wu', linetype='Ecosystem')
plt = plt + scale_colour_manual(values=cbPalette) + scale_fill_manual(values=cbPalette)
plt = plt + scale_y_log10() + scale_x_log10()
plt = plt + geom_abline(intercept=-1, slope=-7/3, linetype='dashed')
plt = plt + facet_wrap('halfhour', nrow=2)
print(plt)
plt.save(graphs_path + 'cospectra_wu.pdf', width=1.6*15, height=1.6*7.5, units='cm', dpi=600)
plt.save(graphs_path + 'cospectra_wu.png', width=1.6*15, height=1.6*7.5, units='cm', dpi=600)

In [ ]:
plt = ggplot(df)
plt = plt + geom_point(aes(x='natural_frequency', y='f_nat*cospec(w_v)/cov(w_v)', colour='Ecosystem'), size=0.5)
plt = plt + theme_bw()
plt = plt + labs(x='Frequency', y='Cospectra wv', linetype='Ecosystem')
plt = plt + scale_colour_manual(values=cbPalette) + scale_fill_manual(values=cbPalette)
plt = plt + scale_y_log10() + scale_x_log10()
plt = plt + geom_abline(intercept=-1, slope=-7/3, linetype='dashed')
plt = plt + facet_wrap('halfhour', nrow=2)
print(plt)
plt.save(graphs_path + 'cospectra_wv.pdf', width=1.6*15, height=1.6*7.5, units='cm', dpi=600)
plt.save(graphs_path + 'cospectra_wv.png', width=1.6*15, height=1.6*7.5, units='cm', dpi=600)